In [1]:
import cv2, time
from datetime import datetime

#capture the video 0 from web-cam and path for pre video
video = cv2.VideoCapture(0)


first_frame = None
status_list=[None,None]
times=[]
while True:          #while check is true
    check,frame = video.read()       #reading the video
    status=0
    print(check)                     #true or false for camera acess
    print(frame)
     #in open cv picture is read by BGR
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   #convert color to gray but no data loss average intensity
    gray=cv2.GaussianBlur(gray,(21,21),0)            #gaussian blur to smooth images (21,21) is the window size

    if first_frame is None:
        first_frame=gray                          #it is assigned for only 1st frame 
        continue

    delta_frame=cv2.absdiff(first_frame,gray)    #absolute difference of frames  |x-y|
    thresh_frame=cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]   #p<t black,else white
    thresh_frame=cv2.dilate(thresh_frame, None, iterations=2)     #dilation makes the white part prominent
                                                                  #erosion makes the black region prominent
    
    #findContours helps to return the boundary of the image. return list of list.
    #return the effective values to detect the object.(chain_external)
    (cnts,_)=cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in cnts:
        if cv2.contourArea(contour) < 10000:      #find the area enclosing the object
            continue

        status = 1               #it is always 1 when object is detected.
        (x, y, w, h)=cv2.boundingRect(contour)  #makes a rectangle enclosing all the point
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)    #draws a rectangle of green color, 3 is for thickness

    status_list.append(status)         #appends the status int ot list 1 for detected 0 for not detected

    if status_list[-1]==1 and status_list[-2]==0:    #cheking if object is coming(entry)
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:    #checking if object is going and was present(exit)
        times.append(datetime.now())

    cv2.imshow("Gray ", gray)                    #show function for the display
    cv2.imshow("Delta ", delta_frame)
    cv2.imshow("Threshold ", thresh_frame)
    cv2.imshow("Color Frame",frame)

    key = cv2.waitKey(1)

    if key == ord('q'):           #terminating condtion
        if status==1:
            times.append(datetime.now())
        break
        
print (status_list)      
print(times)

video.release()
cv2.destroyAllWindows()      #destroy all the caches


False
None


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
